In [1]:
import cv2
import numpy as np
import cv2.aruco
from tqdm import tqdm
import plotly.graph_objects as go

In [2]:
arucoDict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_50)
detector = cv2.aruco.ArucoDetector(arucoDict)

marker_size = 0.141
marker_points = np.array([[-marker_size / 2,  marker_size / 2, 0],
                          [ marker_size / 2,  marker_size / 2, 0],
                          [ marker_size / 2, -marker_size / 2, 0],
                          [-marker_size / 2, -marker_size / 2, 0]], dtype=np.float32)


mtx = np.array([[1.23472227e+03, 0.00000000e+00, 6.38814655e+02],
                [0.00000000e+00, 1.23122213e+03, 3.37257425e+02],
                [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

dist = np.zeros((1, 5))

In [3]:
def pose_esitmation(frame):
    corners, ids, rejected_img_points = detector.detectMarkers(frame)
    if len(corners) > 0:
        for i in range(0, len(ids)):
            nada, rvec, tvec = cv2.solvePnP(marker_points, corners[i][0], mtx, dist, False, cv2.SOLVEPNP_IPPE_SQUARE)

            aruco_pose = np.eye(4)
            aruco_rot_matrix, _ = cv2.Rodrigues(rvec)
            aruco_pose[:3, :3] = aruco_rot_matrix
            aruco_pose[:3, 3:] = np.array(tvec)

            # pose_wrt_aruco = np.linalg.inv(aruco_pose)
            # pose_wrt_aruco = np.array([[1,  0,  0, 0],
            #                            [0, -1,  0, 0],
            #                            [0,  0, -1, 0],
            #                            [0,  0,  0, 1]]) @ pose_wrt_aruco

        return aruco_pose, corners[0][0]
    return None

In [4]:
cap = cv2.VideoCapture('just_aruco.mp4')

frameCount = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frameWidth = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frameHeight = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

buf = np.empty((frameCount, frameHeight, frameWidth, 3), np.dtype('uint8'))

fc = 0
ret = True

while (fc < frameCount  and ret):
    ret, buf[fc] = cap.read()
    fc += 1

cap.release()

In [5]:
positions = []
for i in tqdm(range(len(buf))):
    pose = pose_esitmation(buf[i])
    if pose is not None:
        positions.append(pose[0][:3, 3])

positions = np.array(positions)

  0%|          | 0/309 [00:00<?, ?it/s]

100%|██████████| 309/309 [00:06<00:00, 50.77it/s]


In [6]:
positions = np.array(positions)

In [7]:
keep_candidates = []
for i in range(len(positions)-1):
    speed = np.linalg.norm(positions[i+1] - positions[i]) / 0.033 # 30 fps
    if speed < 0.8:
        keep_candidates.append(i)
        
kept_positions = positions[keep_candidates]
# kept_positions = positions

In [8]:
fig = go.Figure(data=[go.Scatter3d(x=kept_positions.T[0], y=kept_positions.T[1], z=kept_positions.T[2],
                                    marker_size=5, text=np.arange(0, len(kept_positions)), line=dict(
        color='darkblue',
        width=2
    )
)])

fig.show()